In [2]:
import aiohttp

# Add parent dir to path
import os,sys
import io

sys.path.insert(0,os.environ['PWD'])

import netaddr

from rpki_analysis.delegated_stats import read_delegated_stats

In [3]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/pub/stats/ripencc/nro-stats/20240109/nro-delegated-stats') as resp:
        data = read_delegated_stats(io.StringIO(await resp.text()))
        

In [25]:
data.dtypes

rir                  category
country              category
afi                  category
prefix                 object
prefix_size             int64
date           datetime64[ns]
status               category
uuid                   object
category             category
resource               object
dtype: object

In [32]:
resources_by_uuid = data.loc[data.afi != 'asn'].groupby(['uuid']) \
  .agg({'resource': lambda grp: netaddr.IPSet(grp)})

In [37]:
resources_by_uuid.resource.apply(lambda x: list(map(str, x.iter_cidrs())))

uuid
00004a29a9bbf62af6b8e5b91b760abc                                        [192.149.81.0/24]
000076699efb0f677b8a77052cacd0eb                                        [204.68.168.0/24]
0002bdbb-85df-4f43-a83b-3dfe3879b7dd                   [185.119.204.0/22, 2a06:8c00::/29]
0003670b-77cd-4f08-949a-e892c9d63fe2                                   [194.117.228.0/23]
00038785-5f1a-490a-8a6d-481377eed99b                                    [194.242.55.0/24]
                                                              ...                        
ffffb4f7-78b5-4c40-b2e0-3727baa46c82                                   [185.226.136.0/22]
iana                                    [160.20.241.0/24, 160.20.245.0/24, 192.47.28.0...
ietf                                    [0.0.0.0/8, 10.0.0.0/8, 100.64.0.0/10, 127.0.0...
lacnic                                  [45.6.236.0/22, 45.68.33.0/24, 45.68.52.0/22, ...
ripencc                                 [5.63.16.0/21, 45.10.36.0/22, 45.15.41.0/24, 4...
Name:

In [44]:
resources_by_uuid['ipv4_count'] = resources_by_uuid.resource.apply(lambda res: sum([cidr.size if cidr.version == 4 else 0 for cidr in res.iter_cidrs()]))

In [49]:
resources_by_uuid['ipv4_count'].reset_index().sort_values(['ipv4_count'])

,uuid,ipv4_count
23045,314288,0
34352,51b1303c2cece01e6b539e51dbd817b1,0
52887,94a1cd3f-99bc-4635-8d4a-84a4a6fbbe06,0
12280,215638,0
69858,A9248A06,0
...,...,...
28010,3a7b40ab58f6de4849b12c87e5806b7c,70056448
32420,4a8a91b5b89d3f900098ebf73ca0b118,77101824
100909,e220ed812a4c17f6a76b9b930dae7f50,80084224
90582,bb474b75b6f23182ffa56daf1cf9ec23,221958912
